## Scraping Zillow.com to analyze housing price in New York City 

My goal here is to collect housing prices for both rental and sale in New York city. I looked at three major real estate website including Trulia, Zillow, and StreetEasy. Comparing to the other two websites, StreetEasy gives the most information on the searching results page and the format of each listing is very consistent, which is great for the purpose of web-scraping.<br\ >
<a href="http://zillow.com/">
<img "StreetEasy" src="map/streetEasy_logo.jpg" height="30px" width="150px"/></a><br\ >

Web scraping is done using the beautifulsoup package in Python. I created two functions that can loop through all the pages of searching results, and also empty strings to store results. Below are the steps I took to scrape StreetEasy:
1. Analyzing the HTML page: HTML code of a web page can be viewed by right click and selecting 'Inspect'. This helps us identifying the HTML tags of the information to be scraped
2. Making the soup!: It is important to select the correct parser for your data type. I used HTML parser.
3. Navigating the parse tree and iterate through tags: once the soup is made, we have the HTML code in Python. We can then find our desired information by searching through HTML tags.

In [2]:
import os
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import lxml
import numbers
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import random

In [3]:
req_headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-encoding': 'gzip, deflate, br',
    'Accept-language': 'en-US,en;q=0.8',
    'Upgrade-insecure-requests': '1',
    'User-agent': UserAgent().random,
}

#base_url = "https://www.zillow.com/homes/for_sale/"
base_url = "https://streeteasy.com/for-sale/"
urls = []

city = 'nyc'
url1 = base_url +city+'/'
urls.append(url1)

# Add all pages
for i in range(2,2):
    dom = base_url + city + '/' + 'page_' + str(i) #streeteasy
    #dom = base_url + city + '/' + str(i) '_p' +'/' #zillow
    if dom not in urls:
        urls.append(dom)

print(urls)


['https://streeteasy.com/for-sale/nyc/']


In [ ]:
def soups(data):
    with requests.Session() as s:
        r = s.get(data, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        print(soup.prettify())  # Corrected line: Print prettified HTML content from the soup object
    return soup


for url in urls:
    htmls=soups(url)
    print(htmls)
# Call soup function and store output in a list
#lst = []

#for url in urls:
#    htmls = soups(url)

#print(len(urls))

In [ ]:
# Initialize a list to store listing information
def parse_soup(soup):
    listings = []

    # Find all listing cards on the page
    listing_cards = soup.find_all(class_="listingCard")

    for card in listing_cards:
        # Iterate through each listing card and extract relevant information
            # Extract building type and neighborhood
        building_info = card.find(class_="listingCardLabel")
        building_type_neighborhood = building_info.text.strip()

        # Extract address
        address = card.find('address', class_='listingCard-addressLabel listingCard-upperShortLabel').get_text().strip()

        # Extract price
        price = card.find('span', class_='price listingCard-priceMargin').get_text().strip()

        # Extract bed, bathroom details, and size
        description = card.find('div', class_='description').text
        bedrooms = card.find('div', class_='bedrooms').text
        bathrooms = card.find('div', class_='bathrooms').text
        size = card.find('div', class_='size').text

        # Extract amenities match
        amenities_list = card.find('ul', class_='amenities').find_all('li')
        amenities = [amenity.text for amenity in amenities_list]

        # property url
        url = card.find('a', class_='listingCard-link jsCardLinkGA featured-link-to-hdp')['href']

        # Create a dictionary to store the extracted information
        extracted_listings = {
            "address": address,
             "building_type_neighborhood": building_type_neighborhood,
            "size (sq. ft.)": size,
            "price": price,
            "# bedrooms": bedrooms,
            "# bathrooms": bathrooms,
            "amenities": amenities,
            "description": description,
            "url": url
        
        }
        # Append the dictionary to the list
        listings.append(extracted_listings)

    return listings

        # Append the listing dictionary to the list of listings
        
            #address = data.find_all(class_= 'list-card-addr') zillow
            #price = list(data.find_all(class_='list-card-price')) zillow
            #beds = list(data.find_all("ul", class_="list-card-details")) zillow
            #last_updated = data.find_all('div', {'class': 'list-card-top'}) zillow

        

# Example usage:
# Define the URL from which you want to fetch the HTML content
#url = "your_url_here"

# Fetch the HTML content from the URL
#soup = soups(url)

# Parse the HTML content and extract listing information from JSON-LD data
listings = parse_soup(soup)

# Create a pandas DataFrame from the extracted listing information
df = pd.DataFrame(listings)

# Print the DataFrame
print(df)

In [7]:
import pandas as pd
import openpyxl

def soups(data):
    with requests.Session() as s:
        r = s.get(data, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        #print(soup.prettify())  # Corrected line: Print prettified HTML content from the soup object
    return soup
def parse_soup(soup):
    listings = []

    # Find all listing cards on the page
    listing_cards = soup.find_all(class_="listingCard")

    for card in listing_cards:
        # Iterate through each listing card and extract relevant information
            # Extract building type and neighborhood
        building_info = card.find(class_="listingCardLabel")
        building_type_neighborhood = building_info.text.strip()

        # Extract address
        address = card.find('address', class_='listingCard-addressLabel listingCard-upperShortLabel').get_text().strip()

        # Extract price
        price = card.find('span', class_='price listingCard-priceMargin').get_text().strip()

        # Extract bed information
        bed_elem = card.find('span', class_='listingDetailDefinitionsIcon--bed')
        beds = bed_elem.find_next_sibling('span', class_='listingDetailDefinitionsText').text.strip() if bed_elem else None

        # Extract bathroom information
        bath_elem = card.find('span', class_='listingDetailDefinitionsIcon--bath')
        baths = bath_elem.find_next_sibling('span', class_='listingDetailDefinitionsText').text.strip() if bath_elem else None
   
        size_elem = card.find('span', class_='listingDetailDefinitionsIcon--measure')
        size = size_elem.find_next_sibling('span', class_='listingDetailDefinitionsText').text.strip() if size_elem else None

        # Extract amenities match
    #    amenities_list = card.find('ul', class_='amenities').find_all('li')
    #    amenities = [amenity.text for amenity in amenities_list]

        # Extract property URL
        url_element = card.find('a', class_='listingCard-link jsCardLinkGA')
        url = url_element.get('href') if url_element else 'Property URL not found.'


        # Create a dictionary to store the extracted information
        extracted_listings = {
            "Address": address,
            "Building_type_neighborhood": building_type_neighborhood,
            "Size (sq. ft.)": size,
            "Price": price,
            "Bedrooms #": beds,
            "Bathrooms # ": baths,
        #    "amenities": amenities,
            "Url": url
        
        }
        # Append the dictionary to the list
        listings.append(extracted_listings)

    return listings

# Call the parse_soup function to extract listing information
for url in urls:
    soup = soups(url)
    listings = parse_soup(soup)
    df = pd.DataFrame(listings)

    # Display DataFrame with revised structure and formatting
    pd.set_option('display.max_colwidth', None)
    print(df)
    # Export DataFrame to Excel file
    
    excel_file_name = "listings.xlsx"
    df.to_excel(excel_file_name, index=False)
    print(f"DataFrame exported to {excel_file_name}")

                          Address        Building_type_neighborhood  \
0        2286 Cropsey Avenue #18C                          Featured   
1   2351 Adam Clayton Powell #201                          Featured   
2           66 Madison Avenue #4E         Open House, Tue 5:30-6:30   
3               89-15 96th Street               Verified up to date   
4   500 West 18th Street EAST-11F                         Sponsored   
5             36 Remsen Street #2               Verified up to date   
6              2373 Broadway #511              Open House, Sun 12-1   
7             56 Mc Arthur Avenue               Verified up to date   
8       1020 Grand Concourse #11S  Contains \n              3D Tour   
9             138 Randolph Avenue               Verified up to date   
10            1021 Boulevard East               Verified up to date   
11          21-68 35th Street #5D               Verified up to date   
12      235 East 22nd Street #10I               Open House, Sun 2-3   
13    

In [ ]:
zillow_df = pd.DataFrame()

# Get our data from each url and put it into a list of dataframes
df_list = []
for soup in lst:
    new_df = my_soup(soup)
    df_list.append(new_df)
    
# Combine the list of datasets into our new dataframe
zillow_df = pd.concat(df_list)
zillow_df.reset_index(inplace=True)
zillow_df = zillow_df.drop('index', axis=1)

In [ ]:
print(zillow_df.shape)
zillow_df.head()

## Data Manipulation

For some listings the information on number of bedroom, number of bathroom, and apartment size is incomplete or mixed up. I performed data manipulation to fix the mistaken values and clean up the extra symbols such as comma and dollar sign. <br\ >
Finally, I have two data sets containing the housing information for apartments for rent and apartments for sale. My for sale data set has 8,456 rows and 8 columns, and the for rent data set has 20,988 rows and 7 columns

In [ ]:
import pandas as pd
import numpy as np


#is the apartment furnished?
cond=data['bed']=='Furnished'
data.loc[cond,'furnished']=1
data.loc[cond,'bed']=''

#move from size to bath
cond=[]
for i in data['size']:
    if(i==''):
        cond.append(False)
    else:
        cond.append(i.split(" ")[1] in ('bath','baths'))
data.loc[cond,'bath']=data.loc[cond,'size'] 
data.loc[cond,'size']=''

#move from bed to bath
cond=[]
for i in data['bed']:
    if(i=='' or i=='Furnished' or i=='studio'):
        cond.append(False)
    else:
        cond.append(i.split(" ")[1] in ('bath','baths'))
data.loc[cond,'bath']=data.loc[cond,'bed'] 
data.loc[cond,'bed']=''

#move from bath to bed
cond=[]
for i in data['bath']:
    if(i==''):
        cond.append(False)
    else:
        if(len(i.split(" "))==1):
            cond.append(True)
        else:
            if(i.split(" ")[1] in ('bath','baths')):
                cond.append(False)
            else:
                cond.append(True)
data.loc[cond,'bed']=data.loc[cond,'bath'] 
data.loc[cond,'bath']=''

#move from bed to size
cond=[]
for i in data['bed']:
    if(i=='' or i=='studio'):
        cond.append(False)
    else:
        if(i.split(" ")[1] in ('bed','beds')):
            cond.append(False)
        else:
            cond.append(True)
data.loc[cond,'size']=data.loc[cond,'bed'] 
data.loc[cond,'bed']=''


#replace blank with nan
data=data.applymap(lambda x: np.nan if x=='' else x)

#data
data.to_csv('rent.csv',encoding='utf-8')

In [ ]:
#size to numeric
cond=data['size'].isnull()
for i in range(0,len(cond)):
    if (not cond[i]):
        data.loc[i,'size']=int(data['size'][i].split(" ")[0].replace(',',''))
#bath to numeric
cond=data['bath'].isnull()
for i in range(0,len(cond)):
    if (not cond[i]):
        data.loc[i,'bath']=float(data['bath'][i].split(" ")[0].replace('+',''))
#bed to numeric
cond=data['bed'].isnull()
data['bed']=data['bed'].replace('studio','0 bed')
for i in range(0,len(cond)):
    if (not cond[i]):
        data.loc[i,'bed']=float(data['bed'][i].split(" ")[0].replace(',','').replace('+',''))
#remove dollar sign
data['price']=[int(i.replace('$','').replace(',','')) for i in data['price']]

data.to_csv('rent_2.csv')